In [ ]:
import pandas as pd
import numpy as np
import sqlite3

In [ ]:
df_olist_customers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_customers_dataset.csv')
df_olist_sellers = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_sellers_dataset.csv')
df_olist_order_reviews= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_reviews_dataset.csv')
df_olist_order_items= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_items_dataset.csv')
df_olist_products= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_products_dataset.csv')
df_olist_geolocation= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_geolocation_dataset.csv')
df_product_category_name_translation= pd.read_csv('/kaggle/input/brazilian-ecommerce/product_category_name_translation.csv')
df_olist_orders = pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_orders_dataset.csv')
df_olist_order_payments= pd.read_csv('/kaggle/input/brazilian-ecommerce/olist_order_payments_dataset.csv')

df_olist_customers.head()

from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

# export the dataframe as a table 'playstore' to the sqlite engine
df_olist_customers.to_sql("olist_customers", con =engine)
df_olist_sellers.to_sql("olist_sellers", con =engine)
df_olist_order_reviews.to_sql("olist_order_reviews", con =engine)
df_olist_order_items.to_sql("olist_order_items", con =engine)
df_olist_products.to_sql("olist_products_dataset", con =engine)
df_olist_geolocation.to_sql("olist_geolocation", con =engine)
df_product_category_name_translation.to_sql("product_category_name_translation", con =engine)
df_olist_orders.to_sql("olist_orders", con =engine)
df_olist_order_payments.to_sql("olist_order_payments", con =engine)
df_olist_order_payments.head()

In [ ]:
sql='''

Select * from olist_customers
limit 5


''';


df_sql = pd.read_sql_query(sql,con=engine)
df_sql.head()

In [ ]:
# SQL query to count and calculate percentage of 5-star orders in January 2018
sql = '''
SELECT
    COUNT(*) AS count_5_star_orders,
    ROUND( (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM olist_orders o
                                WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01')), 2) AS percentage_5_star_orders
FROM olist_orders o
JOIN olist_order_reviews r ON o.order_id = r.order_id
WHERE strftime('%Y-%m', o.order_purchase_timestamp) = '2018-01'
  AND r.review_score = 5;
'''

df_result = pd.read_sql_query(sql, con=engine)
df_result

In [ ]:
# SQL query to analyze customer purchase trend year-on-year
sql = '''
SELECT
    strftime('%Y', o.order_purchase_timestamp) AS purchase_year,
    COUNT(DISTINCT c.customer_unique_id) AS unique_customers
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
GROUP BY purchase_year
ORDER BY purchase_year;
'''

df_purchase_trend = pd.read_sql_query(sql, con=engine)
df_purchase_trend

In [ ]:
# SQL query to calculate average order values of customers
sql = '''
SELECT
    c.customer_unique_id,
    AVG(oi.price) AS avg_order_value
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_order_items oi ON o.order_id = oi.order_id
GROUP BY c.customer_unique_id
ORDER BY avg_order_value DESC;
'''

df_avg_order_values = pd.read_sql_query(sql, con=engine)
df_avg_order_values.head()

In [ ]:
# SQL query to find the top 5 cities with the highest revenue from 2016 to 2018
sql = '''
SELECT
    g.geolocation_city AS city,
    SUM(oi.price) AS total_revenue
FROM olist_orders o
JOIN olist_order_items oi ON o.order_id = oi.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY g.geolocation_city
ORDER BY total_revenue DESC
LIMIT 5;
'''

df_top_cities = pd.read_sql_query(sql, con=engine)
df_top_cities

In [ ]:
# SQL query to create a state-wise revenue table between 2016 to 2018
sql = '''
SELECT
    g.geolocation_state AS state,
    SUM(oi.price) AS total_revenue
FROM olist_orders o
JOIN olist_order_items oi ON o.order_id = oi.order_id
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
WHERE strftime('%Y', o.order_purchase_timestamp) BETWEEN '2016' AND '2018'
GROUP BY g.geolocation_state
ORDER BY total_revenue DESC;
'''

df_state_wise_revenue = pd.read_sql_query(sql, con=engine)
df_state_wise_revenue

In [ ]:
# SQL query to identify top successful sellers in terms of goods sold, revenue, customer count, and 5-star ratings
sql = '''
SELECT
    s.seller_id,
    COUNT(DISTINCT oi.order_id) AS total_orders,
    SUM(oi.price) AS total_revenue,
    COUNT(DISTINCT o.customer_id) AS unique_customers,
    SUM(CASE WHEN r.review_score = 5 THEN 1 ELSE 0 END) AS five_star_ratings
FROM olist_order_items oi
JOIN olist_orders o ON oi.order_id = o.order_id
JOIN olist_sellers s ON oi.seller_id = s.seller_id
JOIN olist_order_reviews r ON o.order_id = r.order_id
GROUP BY s.seller_id
ORDER BY total_revenue DESC, five_star_ratings DESC, total_orders DESC, unique_customers DESC
LIMIT 10;
'''

df_top_sellers = pd.read_sql_query(sql, con=engine)
df_top_sellers

In [ ]:
# SQL query to calculate the delivery success rate across different states
sql = '''
SELECT
    g.geolocation_state AS state,
    COUNT(o.order_id) AS total_orders,
    SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) AS delivered_orders,
    ROUND((SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) * 100.0 / COUNT(o.order_id)), 2) AS delivery_success_rate
FROM olist_orders o
JOIN olist_customers c ON o.customer_id = c.customer_id
JOIN olist_geolocation g ON c.customer_zip_code_prefix = g.geolocation_zip_code_prefix
GROUP BY g.geolocation_state
ORDER BY delivery_success_rate DESC;
'''

df_delivery_success_rate = pd.read_sql_query(sql, con=engine)
df_delivery_success_rate

In [ ]:
# SQL query to find the preferred form of payment for different product categories
sql = '''
SELECT
    p.product_category_name,
    p.payment_type,
    COUNT(o.order_id) AS total_orders
FROM olist_order_payments op
JOIN olist_orders o ON op.order_id = o.order_id
JOIN olist_order_items oi ON o.order_id = oi.order_id
JOIN olist_products p ON oi.product_id = p.product_id
GROUP BY p.product_category_name, op.payment_type
ORDER BY p.product_category_name, total_orders DESC;
'''

df_preferred_payment = pd.read_sql_query(sql, con=engine)
df_preferred_payment

In [ ]:
# SQL query to calculate the distance between cities using the Haversine formula
sql = '''
WITH city_coordinates AS (
    SELECT
        g1.geolocation_city AS city1,
        g1.geolocation_lat AS lat1,
        g1.geolocation_lng AS lng1,
        g2.geolocation_city AS city2,
        g2.geolocation_lat AS lat2,
        g2.geolocation_lng AS lng2
    FROM olist_geolocation g1
    JOIN olist_geolocation g2 ON g1.geolocation_city <> g2.geolocation_city
)

SELECT
    city1,
    city2,
    ROUND(6371 *
        acos(
            cos(radians(lat1)) * cos(radians(lat2)) *
            cos(radians(lng2) - radians(lng1)) +
            sin(radians(lat1)) * sin(radians(lat2))
        ), 2) AS distance_km
FROM city_coordinates
ORDER BY distance_km;
'''

df_city_distances = pd.read_sql_query(sql, con=engine)
df_city_distances